In [1]:
from datetime import datetime
from urllib.request import urlopen
import pandas as pd
from bs4 import BeautifulSoup
from utils import compute_age

In [2]:
url = f'https://quotes.toscrape.com/'
page = urlopen(url)
soup = BeautifulSoup(page, 'html.parser')
with open('structure.txt', 'w', encoding='utf8') as f:
	f.write(str(soup))

In [32]:
result = soup.find_all('div', class_='quote')
print(result)

[<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
<span>by <small class="author" itemprop="author">Albert Einstein</small>
<a href="/author/Albert-Einstein">(about)</a>
</span>
<div class="tags">
            Tags:
            <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
<a class="tag" href="/tag/change/page/1/">change</a>
<a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>
<a class="tag" href="/tag/thinking/page/1/">thinking</a>
<a class="tag" href="/tag/world/page/1/">world</a>
</div>
</div>, <div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“It is our choices, Harry, that show what we truly are, far more than our abilities.”</span>
<span>by <small class="author" itemprop="author">J.K.

In [33]:
print(*map(lambda r:r.find('small', class_='author').text+'\n', result))

Albert Einstein
 J.K. Rowling
 Albert Einstein
 Jane Austen
 Marilyn Monroe
 Albert Einstein
 André Gide
 Thomas A. Edison
 Eleanor Roosevelt
 Steve Martin



In [34]:
def get_author_content(result):
	quote = result.find('span', class_='text').text[1:-1]
	name = result.find('small', class_='author').text
	author_about = result.find('a', href=True)['href'] + '/'
	link = r'https://quotes.toscrape.com/' + author_about
	author_page = urlopen(link)
	author_soup = BeautifulSoup(author_page, 'html.parser')
	date = author_soup.find('span', class_='author-born-date').text
	date_format = '%B %d, %Y'
	birthday = datetime.strptime(date, date_format)
	single_quote = [name, link, birthday, quote]
	return single_quote

In [35]:
contents = []
pages = 10
for i in range(1, pages + 1):
	url = f'https://quotes.toscrape.com/page/{i}/'
	page = urlopen(url)
	soup = BeautifulSoup(page, 'html.parser')
	result = soup.find_all('div', class_='quote')
	contents.extend(list(map(get_author_content, result)))

In [36]:
df = pd.DataFrame(contents, columns=['name', 'link', 'birthday', 'quote'])
ages = list(map(lambda x, y: compute_age(x, y), df.name, df.birthday))
df.insert(3, column='age', value=ages)

,name,link,birthday,quote
0,Albert Einstein,https://quotes.toscrape.com//author/Albert-Ein...,1879-03-14,The world as we have created it is a process o...
1,J.K. Rowling,https://quotes.toscrape.com//author/J-K-Rowling/,1965-07-31,"It is our choices, Harry, that show what we tr..."
2,Albert Einstein,https://quotes.toscrape.com//author/Albert-Ein...,1879-03-14,There are only two ways to live your life. One...
3,Jane Austen,https://quotes.toscrape.com//author/Jane-Austen/,1775-12-16,"The person, be it gentleman or lady, who has n..."
4,Marilyn Monroe,https://quotes.toscrape.com//author/Marilyn-Mo...,1926-06-01,"Imperfection is beauty, madness is genius and ..."


In [ ]:
df['length'] = df['quote'].apply(lambda x: len(x.split()))

In [ ]:
df.head()

In [ ]:
df.to_csv('./data/Quotes.csv', index=False)

